**Bash_Scripting_Assignment**

Task 1: File Management Script

Write a Bash script that
- Creates a directory named "backup" in the user's home directory^
- Copies all .txt files from the current directory into the "backup" directory^
- Appends the current date and time to the filenames of the copied files

*Ans.*

In [ ]:
```bash
# 1. Define the backup directory path (in user's home directory)
BACKUP_DIR="$HOME/backup"

# Create the directory if it doesn't exist (-p avoids errors if it exists)
mkdir -p "$BACKUP_DIR"

# Get the current date and time for the timestamp (Format: YYYY-MM-DD_HH-MM-SS)
TIMESTAMP=$(date +"%Y-%m-%d_%H-%M-%S")

# 2. Loop through all .txt files in the current directory
for file in *.txt; do
    # Check if the file actually exists (prevents errors if no .txt files are found)
    if [ -f "$file" ]; then

        # Extract the filename without the extension (e.g., 'file1' from 'file1.txt')
        filename=$(basename "$file" .txt)

        # 3. Create the new filename with the timestamp appended
        new_name="${filename}_${TIMESTAMP}.txt"

        # Copy the file to the backup directory with the new name
        cp "$file" "$BACKUP_DIR/$new_name"

        # Optional: Print a confirmation message
        echo "Backed up $file to $new_name"
    fi
done
```

Task 2: System Health Check

Create a script that
- Checks the system’s CPU and memory usage
- Reports if the CPU usage is above 80% or if the available memory is below 20%
- Logs the results to a file named system_health.log

*Ans.*

In [ ]:
#!/bin/bash

# Define thresholds and log file
CPU_THRESHOLD=80
MEM_THRESHOLD=20
LOG_FILE="./system_health.log"

# Function to write timestamped messages to the log file
log_message() {
    echo "[$(date +'%Y-%m-%d %H:%M:%S')] $1" >> "$LOG_FILE"
}

# Get CPU Usage (extracts idle time and calculates used time)
CPU_IDLE=$(top -bn1 | grep "Cpu(s)" | awk '{print $8}' | cut -d',' -f1)
CPU_USED=$((100 - (CPU_IDLE)))

# Get Memory Available Percentage
MEM_TOTAL_MB=$(free -m | grep "Mem:" | awk '{print $2}')
MEM_AVAILABLE_MB=$(free -m | grep "Mem:" | awk '{print $7}')
MEM_AVAILABLE_PERCENT=$((MEM_AVAILABLE_MB * 100 / MEM_TOTAL_MB))

ALERT_STATUS="OK"

# Log current metrics
log_message "--- Metrics: CPU ${CPU_USED}%, Mem Avail ${MEM_AVAILABLE_PERCENT}% ---"

# Check and alert for CPU
if [ "$CPU_USED" -ge "$CPU_THRESHOLD" ]; then
    ALERT_STATUS="ALERT"
    log_message "!! ALERT: High CPU Usage: ${CPU_USED}%"
fi

# Check and alert for Memory
if [ "$MEM_AVAILABLE_PERCENT" -le "$MEM_THRESHOLD" ]; then
    ALERT_STATUS="ALERT"
    log_message "!! ALERT: Low Memory Available: ${MEM_AVAILABLE_PERCENT}%"
fi

# Log final status
log_message "System Health Status: $ALERT_STATUS"
log_message "--------------------------------------"

Task 3: User Account Management

Write a script that
- Reads a list of usernames from a file (e.g., user_list.txt)
- Creates a new user for each username
- Generates a random password for each user and saves the username and password to a file named
credentials.txt.

In [ ]:
#!/bin/bash

# Define file paths
USER_LIST_FILE="./user_list.txt"
CREDENTIALS_FILE="./credentials.txt"

# --- Setup ---

# Check if the user list file exists
if [ ! -f "$USER_LIST_FILE" ]; then
    echo "Error: User list file ($USER_LIST_FILE) not found."
    exit 1
fi

# Ensure the credentials file is fresh and secure
# Overwrite it and set restrictive permissions
echo "Username,Password" > "$CREDENTIALS_FILE"
chmod 600 "$CREDENTIALS_FILE"
echo "Credentials will be saved to $CREDENTIALS_FILE (Permissions: 600)"

# --- Main Logic ---

# Read the user list file line by line
while IFS= read -r username; do
    # 1. Create a new user for each username
    if id "$username" &>/dev/null; then
        echo "User $username already exists. Skipping creation."
        continue
    fi

    # 2. Generate a random password (12 characters, alphanumeric, secure)
    # /dev/urandom is a high-quality entropy source
    # tr filters out characters, head takes the first 12 characters
    password=$(head /dev/urandom | tr -dc A-Za-z0-9 | head -c 12)

    # Create the user (-m creates the home directory) and set the password
    # The 'chpasswd' command is a secure way to set passwords via stdin
    useradd -m "$username"
    echo "$username:$password" | chpasswd

    # 3. Save the username and password to the credentials file
    echo "$username,$password" >> "$CREDENTIALS_FILE"

    echo "Successfully created user: $username"

done < "$USER_LIST_FILE"

echo "Script finished. Check $CREDENTIALS_FILE for new credentials."

Task 4: Automated Backup

Create a script that
- Takes a directory path as input from the user
- Compresses the directory into a .tar.gz file
- Saves the compressed file with a name that includes the current date (e.g., backup_2023-08-20.tar.gz).

In [ ]:
#!/bin/bash

# 1. Check if directory path is provided
if [ -z "$1" ]; then
    echo "Error: Usage: $0 /path/to/directory"
    exit 1
fi

TARGET_DIR="$1"

# Check if the directory exists
if [ ! -d "$TARGET_DIR" ]; then
    echo "Error: Directory '$TARGET_DIR' not found."
    exit 1
fi

# Get the directory name and current date
DIR_NAME=$(basename "$TARGET_DIR")
CURRENT_DATE=$(date +"%Y-%m-%d")

# 2. & 3. Define the output filename
BACKUP_FILENAME="backup_${DIR_NAME}_${CURRENT_DATE}.tar.gz"

echo "Starting backup of: $TARGET_DIR -> $BACKUP_FILENAME"

# Use tar to create (-c), compress with gzip (-z), show verbose output (-v), and specify filename (-f)
tar -czvf "$BACKUP_FILENAME" "$TARGET_DIR"

# Check the tar command exit status
if [ $? -eq 0 ]; then
    echo "Backup successful! Archive created."
else
    echo "Error: Backup failed."
fi

Task 5: Simple To-Do List

Create a Bash script that
- Implements a simple command-line to-do list
- Allows the user to add tasks, view tasks, and remove tasks
- Saves the tasks to a file (e.g., todo.txt)

In [ ]:
#!/bin/bash

TODO_FILE="./todo.txt"

# Ensure file exists
[ ! -f "$TODO_FILE" ] && touch "$TODO_FILE"

# Function to Add a Task
add_task() {
    read -p "Enter the new task: " task_description
    [ -n "$task_description" ] && echo "[ ] $task_description" >> "$TODO_FILE" && echo "Task added." || echo "Task cannot be empty."
}

# Function to View Tasks
view_tasks() {
    echo "--- Current To-Do List ---"
    [ -s "$TODO_FILE" ] && cat -n "$TODO_FILE" || echo "List is empty."
    echo "--------------------------"
}

# Function to Remove a Task
remove_task() {
    view_tasks
    if [ -s "$TODO_FILE" ]; then
        read -p "Enter task number to remove: " num
        # Input validation and deletion using sed
        if [[ "$num" =~ ^[0-9]+$ ]] && [ "$num" -ge 1 ] && [ "$num" -le $(wc -l < "$TODO_FILE") ]; then
            sed -i "${num}d" "$TODO_FILE"
            echo "Task $num removed successfully."
        else
            echo "Error: Invalid task number."
        fi
    fi
}

# Main Script Menu
while true; do
    echo -e "\n--- To-Do List Manager ---\n1. Add Task\n2. View Tasks\n3. Remove Task\n4. Exit"
    read -p "Enter choice (1-4): " choice
    case "$choice" in
        1) add_task ;;
        2) view_tasks ;;
        3) remove_task ;;
        4) echo "Exiting. Goodbye!"; break ;;
        *) echo "Invalid choice." ;;
    esac
done

Task 6: Automated Software Installation

Write a script that
- Reads a list of software package names from a file (e.g., packages.txt)
- Installs each package using the appropriate package manager (apt, yum, etc.)
- Logs the installation status of each package

In [ ]:
#!/bin/bash

# Define file paths
PACKAGE_LIST="./packages.txt"
LOG_FILE="./installation_log.txt"

# Ensure the log file is cleared before starting
> "$LOG_FILE"
echo "--- Software Installation Log Start ---" >> "$LOG_FILE"

# --- Function to detect the package manager ---
detect_package_manager() {
    if command -v apt &>/dev/null; then
        echo "apt"
    elif command -v yum &>/dev/null; then
        echo "yum"
    elif command -v dnf &>/dev/null; then
        echo "dnf"
    else
        echo "unknown"
    fi
}

# --- Function to log status ---
log_status() {
    local status="$1"
    local package="$2"
    echo "[$(date +'%Y-%m-%d %H:%M:%S')] $status: $package" | tee -a "$LOG_FILE"
}

# --- Main Script Execution ---

# 1. Check prerequisite file
if [ ! -f "$PACKAGE_LIST" ]; then
    log_status "ERROR" "Package list file ($PACKAGE_LIST) not found."
    exit 1
fi

# Detect the package manager
PKG_MANAGER=$(detect_package_manager)

if [ "$PKG_MANAGER" == "unknown" ]; then
    log_status "FATAL" "Could not detect a supported package manager (apt, yum, or dnf)."
    exit 1
fi

log_status "INFO" "Detected package manager: $PKG_MANAGER"

# Update package list before installation (Good practice)
log_status "INFO" "Running package list update..."
if [ "$PKG_MANAGER" == "apt" ]; then
    sudo "$PKG_MANAGER" update -y
else
    # yum/dnf do not require a separate 'update' command before 'install'
    log_status "INFO" "Update skipped for $PKG_MANAGER."
fi

# 2. & 3. Read list and install each package
# IFS= ensures whitespace in package names is handled correctly
while IFS= read -r package_name; do
    # Skip empty lines and lines starting with #
    if [ -z "$package_name" ] || [[ "$package_name" =~ ^# ]]; then
        continue
    fi

    package_name=$(echo "$package_name" | xargs) # Trim leading/trailing whitespace

    log_status "INFO" "Attempting to install: $package_name"

    # Install the package using the appropriate manager
    # -y flag automatically answers yes to prompts
    if sudo "$PKG_MANAGER" install -y "$package_name" &>/dev/null; then
        log_status "SUCCESS" "$package_name installed."
    else
        log_status "FAILURE" "$package_name installation failed."
    fi

done < "$PACKAGE_LIST"

log_status "INFO" "--- Installation Script Finished ---"
echo "Installation process complete. Check $LOG_FILE for details."

Task 7: Text File Processing

Create a script that
- Takes a text file as input
- Counts and displays the number of lines, words, and characters in the file
- Finds and displays the longest word in the file.

In [ ]:
#!/bin/bash

# --- 1. Take a text file as input ---

if [ -z "$1" ]; then
    echo "Error: Please provide the path to the text file."
    echo "Usage: $0 /path/to/my_log_file.txt"
    exit 1
fi

INPUT_FILE="$1"

# Check if the input file exists and is a readable file
if [ ! -f "$INPUT_FILE" ] || [ ! -r "$INPUT_FILE" ]; then
    echo "Error: File '$INPUT_FILE' not found or is not readable."
    exit 1
fi

echo "--- Processing File: $INPUT_FILE ---"

# --- 2. Count and display lines, words, and characters ---

# Use 'wc' (word count) utility for fast counting
# -l: lines, -w: words, -c: characters (bytes)
COUNTS=$(wc -l -w -c < "$INPUT_FILE")

# wc output is typically formatted as "LINES WORDS CHARACTERS FILENAME"
# We read the output into separate variables for clean display
read -r LINES WORDS CHARACTERS <<< "$COUNTS"

echo "Line Count:   $LINES"
echo "Word Count:   $WORDS"
echo "Character Count: $CHARACTERS"
echo "-----------------------------------"

# --- 3. Find and display the longest word in the file ---

# This uses a pipeline of standard utilities:
LONGEST_WORD=$(cat "$INPUT_FILE" |  # 1. Read the file content
    tr -s '[:space:]' '\n' |       # 2. Replace all whitespace (spaces, tabs, newlines) with a single newline, putting each word on a new line
    grep -o '[[:alnum:]]\+' |      # 3. Filter out punctuation, keeping only alphanumeric words (optional, but cleaner)
    awk '{ print length, $0 }' |   # 4. Prepend the length of the word to the line
    sort -nr |                     # 5. Sort numerically (-n) and in reverse order (-r), placing the longest word at the top
    head -n 1 |                    # 6. Take only the first (longest) line
    awk '{ print $2 }')            # 7. Print only the word itself (the second column)

if [ -n "$LONGEST_WORD" ]; then
    echo "Longest Word Found: **$LONGEST_WORD**"
    echo "Length: ${#LONGEST_WORD}" # Bash parameter expansion to get string length
else
    echo "No words found in the file."
fi